In [1]:
%matplotlib inline
%config InlineBackend.close_figures=False 

from ANEMO import ANEMO
from ANEMO import read_edf

import os
import pickle
import numpy as np

In [2]:
sujets = ['AM','BMC','CS','DC','FM','IP','LB','OP','RS','SR','TN','YK']
times = ['2017-10-23_100057','2017-09-26_095637', '2017-10-03_134421','2017-09-27_161040',
        '2017-10-03_143803','2017-09-28_115250', '2017-09-20_151043','2017-10-26_121823',
        '2017-11-08_094717','2017-11-16_153313', '2017-11-08_150410','2017-11-17_172706']

In [12]:
num_suj = 1

In [13]:
file = os.path.join('data', 'enregistrement_%s_%s.pkl'%(sujets[num_suj], times[num_suj]))
with open(file, 'rb') as fichier :
    param_exp = pickle.load(fichier, encoding='latin1')
resultats = os.path.join('data', 'enregistrement_%s_%s.asc'%(sujets[num_suj], times[num_suj]))
data = read_edf(resultats, 'TRIALID')

Plot = ANEMO.Plot(param_exp)

In [14]:
list_fct = {'velocity':'fct_velocity',
            #'velocity':'fct_velocity_sigmo',
            #'velocity':'fct_velocity_line',
            #'position':'fct_position',
            #'saccade':'fct_saccade'
           }

In [15]:
Full_param_fit = {}
for fct in list_fct.values() :
    #Full_param_fit[fct] = None
    try :
        file = os.path.join('parametre/param_Fit_%s_%s.pkl'%(sujets[num_suj], fct))
        with open(file, 'rb') as fichier :
            Full_param_fit[fct] = pickle.load(fichier, encoding='latin1')
        print(fct, 'OK param')
    except :
        Full_param_fit[fct] = None

fct_velocity OK param


In [16]:
list_delete = {}
for fct in list_fct.keys():
    print(fct, '\n-------------------------------------------')
    try :
        with open('parametre/Delete_list_BadTrials_%s_%s.pkl'%(sujets[num_suj], list_fct[fct]), 'rb') as fichier:
            f = pickle.load(fichier, encoding='latin1')
        print(f)
        list_delete[list_fct[fct]] = f
        
    except : list_delete[list_fct[fct]] = None ; print('No file!')

velocity 
-------------------------------------------
[[1, 7, 25, 30, 81, 156, 179, 187], [57, 66, 71, 80, 97, 126, 128, 129, 137], [19, 30, 33, 79, 96, 118, 123, 137, 157, 167, 190]]


In [17]:
print(list_delete)

{'fct_velocity': [[1, 7, 25, 30, 81, 156, 179, 187], [57, 66, 71, 80, 97, 126, 128, 129, 137], [19, 30, 33, 79, 96, 118, 123, 137, 157, 167, 190]]}


In [18]:
import matplotlib.pyplot as plt

In [19]:
list_param_enre = {}
list_param_enre['fct_velocity'] = ['start_anti', 'a_anti', 'latency', 'tau', 'steady_state']
list_param_enre['fct_velocity_sigmo'] = ['start_anti', 'a_anti', 'latency', 'ramp_pursuit', 'steady_state']
list_param_enre['fct_velocity_line'] = ['start_anti', 'a_anti', 'latency', 'ramp_pursuit', 'steady_state']
list_param_enre['fct_position'] = ['start_anti', 'a_anti', 'latency', 'tau', 'steady_state']
list_param_enre['fct_saccade'] =  ['T0', 't1', 't2', 'tr', 'x_0', 'x1', 'x2', 'tau']

In [20]:
for fct in list_fct.keys():
    
    for block in range(param_exp['N_blocks']) :
        
        trials = list_delete[list_fct[fct]][block]
        print(block, trials)
        
        
        fig_width=15
        fig, axs = plt.subplots(len(trials), 1, figsize=(fig_width, (fig_width*(len(trials)/2)/1.6180)))
        
        for t, trial in enumerate(trials) :
            
            param_fit_trial = {}
            for name in list_param_enre[list_fct[fct]] :
                if type(Full_param_fit[list_fct[fct]][name][block][trial])!= list :
                    if np.isnan(Full_param_fit[list_fct[fct]][name][block][trial]) : param_fit_trial = None ; break
                    else : param_fit_trial[name] = Full_param_fit[list_fct[fct]][name][block][trial]
                else : param_fit_trial[name] = Full_param_fit[list_fct[fct]][name][block][trial]
            #print(param_fit_trial)
            axs[t] = Plot.generate_fig(axs[t], data, trial, block, 
                                     show='fit', show_data='velocity', equation='fct_velocity',
                                     N_blocks=param_exp['N_blocks'], N_trials=param_exp['N_trials'],
                                     time_sup=280, step_fit=2, do_whitening=False,
                                     list_param_enre=None, param_fit=param_fit_trial, inde_vars=None,
                                     before_sacc=5, after_sacc=15, stop_search_misac=None,
                                     filt=None, cutoff=30, sample_rate=1000,
                                     show_pos_sacc=True, plot_detail=None,
                                     show_target=False, show_num_trial=True, write_step_trial=False,
                                     title='', c='k', fig=fig, out=None, report=None,
                                     fig_width=fig_width, t_label=20, t_text=14) 

        
        plt.tight_layout()
        plt.subplots_adjust(hspace=0)
        plt.savefig('parametre/Delete/Delete_list_BadTrials_%s_%s_%s.pdf'%(sujets[num_suj], list_fct[fct], block+1))
        plt.close()

0 [1, 7, 25, 30, 81, 156, 179, 187]
1 [57, 66, 71, 80, 97, 126, 128, 129, 137]
2 [19, 30, 33, 79, 96, 118, 123, 137, 157, 167, 190]
